In [4]:
import cv2
import pytesseract
import numpy as np
import torch

import torchaudio
import moviepy
from moviepy.editor import AudioFileClip
from transformers import pipeline

# تحميل نموذج التعرف على الكلام من OpenAI Whisper
whisper_model = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# تحميل الفيديو
video_path = "AutoRef-video.mp4"
cap = cv2.VideoCapture(video_path)

# إعداد Tesseract OCR لاستخراج النصوص (نتيجة المباراة)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # تعديل حسب نظامك

# دالة لاستخراج النصوص من إطار معين (مثل النتيجة)
def extract_score(frame):
    roi = frame[50:100, 400:600]  # تعديل حسب موقع النتيجة في الفيديو
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    score_text = pytesseract.image_to_string(gray)
    return score_text.strip()

# تحليل الفيديو إطارًا بإطار
frame_count = 0
previous_score = ""

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 30 == 0:  # تحليل كل ثانية (30 إطار في الثانية)
        score = extract_score(frame)
        if score and score != previous_score:
            print(f"⚽ تغيير في النتيجة: {score}")
            previous_score = score

cap.release()

# استخراج الصوت من الفيديو
audio_path = "match_audio.wav"
audio = AudioFileClip(video_path)
audio.write_audiofile(audio_path)

# تحويل الصوت إلى نص باستخدام Whisper
def transcribe_audio(audio_path):
    audio_input, _ = torchaudio.load(audio_path)
    transcription = whisper_model(audio_input.numpy())
    return transcription['text']

# استخراج التعليق الصوتي
commentary_text = transcribe_audio(audio_path)
print("🎙️ التعليق الرياضي:")
print(commentary_text)

# تحليل المشاهد الحماسية بناءً على مستوى الصوت
import librosa

def detect_exciting_moments(audio_path, threshold=0.1):
    y, sr = librosa.load(audio_path)
    energy = librosa.feature.rms(y=y)
    exciting_moments = np.where(energy > threshold)[1]  # اللحظات ذات الصوت العالي
    return exciting_moments

exciting_moments = detect_exciting_moments(audio_path)
print(f"🔥 عدد اللحظات الحماسية المكتشفة: {len(exciting_moments)}")

ModuleNotFoundError: No module named 'moviepy.editor'

In [5]:
# استيراد المكتبات
from ultralytics import YOLO
import cv2
import numpy as np

# تحميل نموذج YOLOv8 المدرب مسبقاً
model = YOLO('yolov8n.pt')  # يمكن تحميل نسخة أكبر مثل yolov8s.pt لتحسين الدقة

# تحميل الفيديو
video_path = 'match_video.mp4'
cap = cv2.VideoCapture(video_path)

# التأكد من فتح الفيديو
if not cap.isOpened():
    print("خطأ في فتح الفيديو")
    exit()

# إعداد المتغيرات
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# إنشاء نافذة العرض
cv2.namedWindow("VAR System", cv2.WINDOW_NORMAL)
cv2.resizeWindow("VAR System", frame_width // 2, frame_height // 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # تصغير الإطار لتسريع المعالجة
    frame = cv2.resize(frame, (960, 540))

    # الكشف باستخدام YOLOv8
    results = model(frame)

    # تخزين مواقع اللاعبين والكرة
    players = []
    ball = None

    # تحليل النتائج
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # تصنيف الكائن
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # إحداثيات الصندوق

            # تصنيف الكائنات بناءً على معرف YOLO:
            # اللاعب = معرفه يكون في "person"، والكرة في معرف "sports ball"
            if cls == 0:  # الشخص
                players.append((x1, y1, x2, y2))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            elif cls == 32:  # الكرة
                ball = (x1, y1, x2, y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # حساب خط التسلل
    if len(players) > 1:
        # تحديد اللاعب الأخير في الدفاع (أقرب لاعب للمرمى)
        players_sorted = sorted(players, key=lambda p: p[0])
        offside_line = players_sorted[-2][0]

        # رسم خط التسلل
        cv2.line(frame, (offside_line, 0), (offside_line, frame.shape[0]), (255, 0, 0), 2)

        # التحقق من التسلل: إذا كان المهاجم متقدماً على آخر مدافع والكرة ليست أمامه
        for p in players_sorted:
            if p[0] > offside_line and (ball is None or ball[0] > offside_line):
                cv2.putText(frame, "Offside!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # عرض الفيديو
    cv2.imshow('VAR System', frame)

    # الخروج عند الضغط على "q"
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\ACER\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:04<00:00, 1.43MB/s]



0: 384x640 7 persons, 342.2ms
Speed: 18.5ms preprocess, 342.2ms inference, 24.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 242.5ms
Speed: 12.6ms preprocess, 242.5ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 220.8ms
Speed: 8.3ms preprocess, 220.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 177.7ms
Speed: 8.5ms preprocess, 177.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 213.1ms
Speed: 9.1ms preprocess, 213.1ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 227.7ms
Speed: 7.9ms preprocess, 227.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 243.7ms
Speed: 7.3ms preprocess, 243.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 202.7ms
Speed: 6.0ms prepr

In [6]:
# استيراد المكتبات
from ultralytics import YOLO
import cv2
import numpy as np

# تحميل نموذج YOLOv8
model = YOLO('yolov8n.pt')  # يمكنك استخدام yolov8s.pt لتحسين الدقة

# تحميل الفيديو الأصلي
video_path = 'match_video.mp4'
cap = cv2.VideoCapture(video_path)

# التأكد من فتح الفيديو
if not cap.isOpened():
    print("خطأ في فتح الفيديو")
    exit()

# إعداد أبعاد الفيديو الأصلي
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))  # عدد الإطارات في الثانية

# إعداد كودك لضغط الفيديو وحفظه
output_path = 'output_var.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # استخدم 'XVID' أو 'MJPG' إذا لم يعمل
out = cv2.VideoWriter(output_path, fourcc, fps, (960, 540))

# إنشاء نافذة العرض
cv2.namedWindow("VAR System", cv2.WINDOW_NORMAL)
cv2.resizeWindow("VAR System", frame_width // 2, frame_height // 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # تصغير الإطار لتسريع المعالجة
    frame = cv2.resize(frame, (960, 540))

    # الكشف باستخدام YOLOv8
    results = model(frame)

    # تخزين مواقع اللاعبين والكرة
    players = []
    ball = None

    # تحليل النتائج
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # تصنيف الكائن
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # إحداثيات الصندوق

            # اللاعب = "person"، والكرة = "sports ball"
            if cls == 0:  # الشخص
                players.append((x1, y1, x2, y2))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # إطار أخضر حول اللاعب
            elif cls == 32:  # الكرة
                ball = (x1, y1, x2, y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # إطار أحمر حول الكرة

    # حساب خط التسلل
    if len(players) > 1:
        # تحديد اللاعب الأخير في الدفاع (أقرب لاعب للمرمى)
        players_sorted = sorted(players, key=lambda p: p[0])
        offside_line = players_sorted[-2][0]

        # رسم خط التسلل
        cv2.line(frame, (offside_line, 0), (offside_line, frame.shape[0]), (255, 0, 0), 2)  # خط أزرق

        # التحقق من التسلل: إذا كان المهاجم متقدماً على آخر مدافع والكرة ليست أمامه
        for p in players_sorted:
            if p[0] > offside_line and (ball is None or ball[0] > offside_line):
                cv2.putText(frame, "Offside!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # عرض الفيديو المباشر
    cv2.imshow('VAR System', frame)

    # حفظ الإطار في الفيديو الجديد
    out.write(frame)

    # الخروج عند الضغط على "q"
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# تحرير الموارد
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"تم حفظ الفيديو المعدل بنجاح في: {output_path}")


0: 384x640 7 persons, 314.1ms
Speed: 74.0ms preprocess, 314.1ms inference, 59.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 465.4ms
Speed: 34.4ms preprocess, 465.4ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 198.2ms
Speed: 14.6ms preprocess, 198.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 230.5ms
Speed: 6.2ms preprocess, 230.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 194.9ms
Speed: 6.9ms preprocess, 194.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 231.3ms
Speed: 6.4ms preprocess, 231.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 172.3ms
Speed: 5.6ms preprocess, 172.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 172.3ms
Speed: 6.9ms prep

In [6]:
# استيراد المكتبات
from ultralytics import YOLO
import cv2
import numpy as np

# تحميل نموذج YOLOv8
model = YOLO('yolov8n.pt')  # يمكنك استخدام yolov8s.pt لتحسين الدقة

# تحميل الفيديو الأصلي
video_path = 'match_video.mp4'
cap = cv2.VideoCapture(video_path)

# التأكد من فتح الفيديو
if not cap.isOpened():
    print("خطأ في فتح الفيديو")
    exit()

# إعداد أبعاد الفيديو الأصلي
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))  # عدد الإطارات في الثانية

# إعداد كودك لضغط الفيديو وحفظه
output_path = 'output_var.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # استخدم 'XVID' أو 'MJPG' إذا لم يعمل
out = cv2.VideoWriter(output_path, fourcc, fps, (960, 540))

# إنشاء نافذة العرض
cv2.namedWindow("VAR System", cv2.WINDOW_NORMAL)
cv2.resizeWindow("VAR System", frame_width // 2, frame_height // 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # تصغير الإطار لتسريع المعالجة
    frame = cv2.resize(frame, (960, 540))

    # الكشف باستخدام YOLOv8
    results = model(frame)

    # تخزين مواقع اللاعبين والكرة
    players = []
    ball = None

    # تحليل النتائج
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # تصنيف الكائن
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # إحداثيات الصندوق

            # اللاعب = "person"، والكرة = "sports ball"
            if cls == 0:  # الشخص
                players.append((x1, y1, x2, y2))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # إطار أخضر حول اللاعب
            elif cls == 32:  # الكرة
                ball = (x1, y1, x2, y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # إطار أحمر حول الكرة

    # حساب خط التسلل
    if len(players) > 1:
        # تحديد اللاعب الأخير في الدفاع (أقرب لاعب للمرمى)
        players_sorted = sorted(players, key=lambda p: p[0])
        offside_line = players_sorted[-2][0]

        # رسم خط التسلل
        cv2.line(frame, (offside_line, 0), (offside_line, frame.shape[0]), (255, 0, 0), 2)  # خط أزرق

        # التحقق من التسلل: إذا كان المهاجم متقدماً على آخر مدافع والكرة ليست أمامه
        for p in players_sorted:
            if p[0] > offside_line and (ball is None or ball[0] > offside_line):
                cv2.putText(frame, "Offside!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # عرض الفيديو المباشر
    cv2.imshow('VAR System', frame)

    # حفظ الإطار في الفيديو الجديد
    out.write(frame)

    # الخروج عند الضغط على "q"
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# تحرير الموارد
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"تم حفظ الفيديو المعدل بنجاح في: {output_path}")


0: 384x640 7 persons, 314.1ms
Speed: 74.0ms preprocess, 314.1ms inference, 59.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 465.4ms
Speed: 34.4ms preprocess, 465.4ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 198.2ms
Speed: 14.6ms preprocess, 198.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 230.5ms
Speed: 6.2ms preprocess, 230.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 194.9ms
Speed: 6.9ms preprocess, 194.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 231.3ms
Speed: 6.4ms preprocess, 231.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 172.3ms
Speed: 5.6ms preprocess, 172.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 172.3ms
Speed: 6.9ms prep

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from mediapipe import solutions

# ==== تحميل YOLO و Mediapipe ====
model = YOLO('yolov8n.pt')
pose = solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# ==== فتح الفيديو ====
video_path = 'match_video.mp4'
cap = cv2.VideoCapture(video_path)
frame_width, frame_height = int(cap.get(3)), int(cap.get(4))

# ==== Kalman Filter لتتبع الكرة ====
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03

# ==== إعدادات ====
ball_trajectory = []  # لتتبع مسار الكرة
team_colors = [(0, 255, 0), (0, 0, 255)]  # ألوان الفرق (أخضر وأحمر)
handball_color = (0, 0, 255)  # لون لمسة اليد
foul_color = (0, 255, 255)  # لون المخالفات
line_color = (255, 255, 0)  # لون خط التسلل

# ==== قراءة الفيديو إطار بإطار ====
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ==== اكتشاف اللاعبين والكرة باستخدام YOLO ====
    results = model(frame)
    players = []
    ball = None

for result in results:
    for box in result.boxes:
        cls = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])

    if cls == 32:  # كرة القدم
        ball = (x1 + x2) // 2, (y1 + y2) // 2
        kalman.correct(np.array(ball, np.float32))
        prediction = kalman.predict()
        ball_trajectory.append((int(prediction[0]), int(prediction[1])))
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, "Ball", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    elif cls == 0:  # لاعب كرة قدم
        players.append((x1, y1, x2, y2))

    # ==== تتبع الكرة عند اختفائها ====
if not ball:
     prediction = kalman.predict()
     ball_trajectory.append((int(prediction[0]), int(prediction[1])))

    # ==== رسم مسار الكرة ====
for i in range(1, len(ball_trajectory)):
    cv2.line(frame, ball_trajectory[i - 1], ball_trajectory[i], (0, 255, 0), 2)

    # ==== كشف الأخطاء (لمسة اليد والفاولات) ====
for (x1, y1, x2, y2) in players:
     player_crop = frame[y1:y2, x1:x2]
     rgb_crop = cv2.cvtColor(player_crop, cv2.COLOR_BGR2RGB)
     pose_results = pose.process(rgb_crop)

if pose_results.pose_landmarks:
        left_hand = pose_results.pose_landmarks.landmark[15]
        right_hand = pose_results.pose_landmarks.landmark[16]

        left_hand_pos = (int(left_hand.x * (x2 - x1)) + x1, int(left_hand.y * (y2 - y1)) + y1)
        right_hand_pos = (int(right_hand.x * (x2 - x1)) + x1, int(right_hand.y * (y2 - y1)) + y1)

            # كشف لمسة اليد
if ball:
    distance_left = np.linalg.norm(np.array(left_hand_pos) - np.array(ball))
    distance_right = np.linalg.norm(np.array(right_hand_pos) - np.array(ball))

    if distance_left < 50 or distance_right < 50:
        cv2.putText(frame, "HANDBALL!", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, handball_color, 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), handball_color, 2)

    # ==== كشف المخالفات ====

for i, (x1, y1, x2, y2) in enumerate(players):
    for j, (x1_2, y1_2, x2_2, y2_2) in enumerate(players):
        if i != j:
            # إذا اقترب لاعبان جدًا من بعضهما
            if abs(x1 - x1_2) < 50 and abs(y1 - y1_2) < 50:
                cv2.putText(frame, "FOUL!", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, foul_color, 2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), foul_color, 2)
                # إذا اقترب لاعبان جدًا من بعضهما
        if abs(x1 - x1_2) < 50 and abs(y1 - y1_2) < 50:
            cv2.putText(frame, "FOUL!", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, foul_color, 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), foul_color, 2)

    # ==== كشف التسلل ====
if len(players) >= 2:
    players.sort(key=lambda p: p[0])  # ترتيب اللاعبين حسب موقعهم الأفقي
    last_defender = players[-2]  # آخر مدافع
    attacker = players[-1]  # أقرب مهاجم للمرمى

        # رسم خط التسلل
cv2.line(frame, (last_defender[0], 0), (last_defender[0], frame_height), line_color, 2)

        # تحقق من التسلل
if attacker[0] > last_defender[0]:
        cv2.putText(frame, "OFFSIDE!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # ==== عرض الفيديو ====
cv2.imshow('Football Analysis', frame)

if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np

# تحميل نموذج YOLOv8
model = YOLO('yolov8n.pt')  # يمكنك استخدام yolov8s.pt لتحسين الدقة

# تحميل الفيديو الأصلي
video_path = 'match_video.mp4'
cap = cv2.VideoCapture(video_path)

# التأكد من فتح الفيديو
if not cap.isOpened():
    print("خطأ في فتح الفيديو")
    exit()

# إعداد أبعاد الفيديو الأصلي
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))  # عدد الإطارات في الثانية

# إعداد كودك لضغط الفيديو وحفظه
output_path = 'output_var_with_cards.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # استخدم 'XVID' أو 'MJPG' إذا لم يعمل
out = cv2.VideoWriter(output_path, fourcc, fps, (960, 540))

# إنشاء نافذة العرض
cv2.namedWindow("VAR System", cv2.WINDOW_NORMAL)
cv2.resizeWindow("VAR System", frame_width // 2, frame_height // 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # تصغير الإطار لتسريع المعالجة
    frame = cv2.resize(frame, (960, 540))

    # الكشف باستخدام YOLOv8
    results = model(frame)

    # تخزين مواقع اللاعبين والكرة
    players = []
    ball = None

    # تحليل النتائج
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # تصنيف الكائن
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # إحداثيات الصندوق

            # اللاعب = "person"، والكرة = "sports ball"
            if cls == 0:  # الشخص
                players.append((x1, y1, x2, y2))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # إطار أخضر حول اللاعب
            elif cls == 32:  # الكرة
                ball = (x1, y1, x2, y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # إطار أحمر حول الكرة

    # حساب خط التسلل
    if len(players) > 1:
        # تحديد اللاعب الأخير في الدفاع (أقرب لاعب للمرمى)
        players_sorted = sorted(players, key=lambda p: p[0])
        offside_line = players_sorted[-2][0]

        # رسم خط التسلل
        cv2.line(frame, (offside_line, 0), (offside_line, frame.shape[0]), (255, 0, 0), 2)  # خط أزرق

        # التحقق من التسلل: إذا كان المهاجم متقدماً على آخر مدافع والكرة ليست أمامه
        for p in players_sorted:
            if p[0] > offside_line and (ball is None or ball[0] > offside_line):
                cv2.putText(frame, "Offside!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # إضافة البطاقات بناءً على التصادمات
    # نعتبر إذا كانت المسافة بين اللاعبين صغيرة جدًا، أنها حالة تدخل قوي
    for i in range(len(players)):
        for j in range(i + 1, len(players)):
            p1 = players[i]
            p2 = players[j]
            distance = np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
            if distance < 100:  # إذا كان التباعد بين اللاعبين أقل من 100 (على سبيل المثال)
                # عرض البطاقة الصفراء أو الحمراء
                if distance < 50:
                    # تدخل قوي جدًا، بطاقة حمراء
                    cv2.putText(frame, "Red Card!", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                    cv2.rectangle(frame, (p1[0], p1[1]), (p1[2], p1[3]), (0, 0, 255), 2)  # إطار أحمر حول اللاعب
                else:
                    # تدخل قوي، بطاقة صفراء
                    cv2.putText(frame, "Yellow Card!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)
                    cv2.rectangle(frame, (p1[0], p1[1]), (p1[2], p1[3]), (0, 255, 255), 2)  # إطار أصفر حول اللاعب

    # عرض الفيديو المباشر
    cv2.imshow('VAR System', frame)

    # حفظ الإطار في الفيديو الجديد
    out.write(frame)

    # الخروج عند الضغط على "q"
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# تحرير الموارد
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"تم حفظ الفيديو المعدل بنجاح في: {output_path}")


0: 384x640 7 persons, 376.1ms
Speed: 36.0ms preprocess, 376.1ms inference, 28.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 235.9ms
Speed: 7.4ms preprocess, 235.9ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 182.8ms
Speed: 5.9ms preprocess, 182.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 216.3ms
Speed: 6.8ms preprocess, 216.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 164.5ms
Speed: 5.4ms preprocess, 164.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 175.9ms
Speed: 5.6ms preprocess, 175.9ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 152.3ms
Speed: 4.6ms preprocess, 152.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 153.3ms
Speed: 7.4ms prepro

In [3]:
from ultralytics import YOLO
import cv2
import numpy as np

# تحميل نموذج YOLOv8
model = YOLO('yolov8n.pt')  # يمكنك استخدام yolov8s.pt لتحسين الدقة

# تحميل الفيديو الأصلي
video_path = 'match_video.mp4'
cap = cv2.VideoCapture(video_path)

# التأكد من فتح الفيديو
if not cap.isOpened():
    print("خطأ في فتح الفيديو")
    exit()

# إعداد أبعاد الفيديو الأصلي
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))  # عدد الإطارات في الثانية

# إعداد كودك لضغط الفيديو وحفظه
output_path = 'output_var_with_cards2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # استخدم 'XVID' أو 'MJPG' إذا لم يعمل
out = cv2.VideoWriter(output_path, fourcc, fps, (960, 540))

# إنشاء نافذة العرض
cv2.namedWindow("VAR System", cv2.WINDOW_NORMAL)
cv2.resizeWindow("VAR System", frame_width // 2, frame_height // 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # تصغير الإطار لتسريع المعالجة
    frame = cv2.resize(frame, (960, 540))

    # الكشف باستخدام YOLOv8
    results = model(frame)

    # تخزين مواقع اللاعبين والكرة
    players = []
    ball = None
    card_displayed = False  # متغير لتتبع إذا تم عرض البطاقة أم لا

    # تحليل النتائج
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # تصنيف الكائن
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # إحداثيات الصندوق

            # اللاعب = "person"، والكرة = "sports ball"
            if cls == 0:  # الشخص
                players.append((x1, y1, x2, y2))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # إطار أخضر حول اللاعب
            elif cls == 32:  # الكرة
                ball = (x1, y1, x2, y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # إطار أحمر حول الكرة

    # حساب خط التسلل
    offside_displayed = False  # متغير لتتبع إذا تم عرض التسلل

    if len(players) > 1:
        # تحديد اللاعب الأخير في الدفاع (أقرب لاعب للمرمى)
        players_sorted = sorted(players, key=lambda p: p[0])
        offside_line = players_sorted[-2][0]

        # رسم خط التسلل
        cv2.line(frame, (offside_line, 0), (offside_line, frame.shape[0]), (255, 0, 0), 2)  # خط أزرق

        # التحقق من التسلل: إذا كان المهاجم متقدماً على آخر مدافع والكرة ليست أمامه
        for p in players_sorted:
            if p[0] > offside_line and (ball is None or ball[0] > offside_line):
                # إذا تم اكتشاف التسلل
                cv2.putText(frame, "Offside!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                offside_displayed = True
                break  # عرض التسلل مرة واحدة فقط

    # إضافة البطاقات بناءً على التصادمات
    for i in range(len(players)):
        for j in range(i + 1, len(players)):
            p1 = players[i]
            p2 = players[j]
            distance = np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
            if distance < 100:  # إذا كان التباعد بين اللاعبين أقل من 100 (على سبيل المثال)
                if distance < 50 and not card_displayed:  # تدخل قوي جدًا، بطاقة حمراء
                    cv2.putText(frame, "Red Card!", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                    card_displayed = True
                    cv2.rectangle(frame, (p1[0], p1[1]), (p1[2], p1[3]), (0, 0, 255), 2)  # إطار أحمر حول اللاعب
                elif distance >= 50 and not card_displayed:  # تدخل قوي، بطاقة صفراء
                    cv2.putText(frame, "Yellow Card!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)
                    card_displayed = True
                    cv2.rectangle(frame, (p1[0], p1[1]), (p1[2], p1[3]), (0, 255, 255), 2)  # إطار أصفر حول اللاعب

    # عرض الفيديو المباشر
    cv2.imshow('VAR System', frame)

    # حفظ الإطار في الفيديو الجديد
    out.write(frame)

    # الخروج عند الضغط على "q"
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# تحرير الموارد
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"تم حفظ الفيديو المعدل بنجاح في: {output_path}")


0: 384x640 7 persons, 190.6ms
Speed: 9.4ms preprocess, 190.6ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 173.3ms
Speed: 5.2ms preprocess, 173.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 435.2ms
Speed: 5.9ms preprocess, 435.2ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 255.2ms
Speed: 10.6ms preprocess, 255.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 174.4ms
Speed: 6.7ms preprocess, 174.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 163.9ms
Speed: 5.5ms preprocess, 163.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 185.8ms
Speed: 5.8ms preprocess, 185.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 157.2ms
Speed: 5.8ms preproc

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# تحميل نموذج YOLOv8
model = YOLO('yolov8n.pt')  # يمكنك استخدام yolov8s.pt لتحسين الدقة

# تحميل الفيديو الأصلي
video_path = 'match_video.mp4'
cap = cv2.VideoCapture(video_path)

# التأكد من فتح الفيديو
if not cap.isOpened():
    print("خطأ في فتح الفيديو")
    exit()

# إعداد أبعاد الفيديو الأصلي
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))  # عدد الإطارات في الثانية

# إعداد كودك لضغط الفيديو وحفظه
output_path = 'output_var_with_cards3.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # استخدم 'XVID' أو 'MJPG' إذا لم يعمل
out = cv2.VideoWriter(output_path, fourcc, fps, (960, 540))

# إنشاء نافذة العرض
cv2.namedWindow("VAR System", cv2.WINDOW_NORMAL)
cv2.resizeWindow("VAR System", frame_width // 2, frame_height // 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # تصغير الإطار لتسريع المعالجة
    frame = cv2.resize(frame, (960, 540))

    # الكشف باستخدام YOLOv8
    results = model(frame)

    # تخزين مواقع اللاعبين والكرة
    players = []
    ball = None

    # متغيرات لتتبع إذا تم عرض البطاقة أو التسلل في هذا الإطار
    offside_displayed = False
    card_displayed = False

    # تحليل النتائج
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # تصنيف الكائن
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # إحداثيات الصندوق

            # اللاعب = "person"، والكرة = "sports ball"
            if cls == 0:  # الشخص
                players.append((x1, y1, x2, y2))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # إطار أخضر حول اللاعب
            elif cls == 32:  # الكرة
                ball = (x1, y1, x2, y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # إطار أحمر حول الكرة

    # حساب خط التسلل
    if len(players) > 1 and not offside_displayed:
        # تحديد اللاعب الأخير في الدفاع (أقرب لاعب للمرمى)
        players_sorted = sorted(players, key=lambda p: p[0])
        offside_line = players_sorted[-2][0]

        # رسم خط التسلل
        cv2.line(frame, (offside_line, 0), (offside_line, frame.shape[0]), (255, 0, 0), 2)  # خط أزرق

        # التحقق من التسلل: إذا كان المهاجم متقدماً على آخر مدافع والكرة ليست أمامه
        for p in players_sorted:
            if p[0] > offside_line and (ball is None or ball[0] > offside_line):
                # إذا تم اكتشاف التسلل
                cv2.putText(frame, "Offside!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                offside_displayed = True
                break  # عرض التسلل مرة واحدة فقط

    # إضافة البطاقات بناءً على التصادمات
    for i in range(len(players)):
        for j in range(i + 1, len(players)):
            p1 = players[i]
            p2 = players[j]
            distance = np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
            if distance < 100:  # إذا كان التباعد بين اللاعبين أقل من 100 (على سبيل المثال)
                if distance < 50 and not card_displayed:  # تدخل قوي جدًا، بطاقة حمراء
                    cv2.putText(frame, "Red Card!", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                    card_displayed = True
                    cv2.rectangle(frame, (p1[0], p1[1]), (p1[2], p1[3]), (0, 0, 255), 2)  # إطار أحمر حول اللاعب
                elif distance >= 50 and not card_displayed:  # تدخل قوي، بطاقة صفراء
                    cv2.putText(frame, "Yellow Card!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)
                    card_displayed = True
                    cv2.rectangle(frame, (p1[0], p1[1]), (p1[2], p1[3]), (0, 255, 255), 2)  # إطار أصفر حول اللاعب

    # عرض الفيديو المباشر
    cv2.imshow('VAR System', frame)

    # حفظ الإطار في الفيديو الجديد
    out.write(frame)

    # الخروج عند الضغط على "q"
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# تحرير الموارد
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"تم حفظ الفيديو المعدل بنجاح في: {output_path}")



0: 384x640 7 persons, 579.2ms
Speed: 108.0ms preprocess, 579.2ms inference, 29.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 363.7ms
Speed: 8.0ms preprocess, 363.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 234.6ms
Speed: 6.5ms preprocess, 234.6ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 184.9ms
Speed: 6.4ms preprocess, 184.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 202.0ms
Speed: 7.4ms preprocess, 202.0ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 185.5ms
Speed: 6.2ms preprocess, 185.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 191.4ms
Speed: 6.0ms preprocess, 191.4ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 211.8ms
Speed: 9.6ms prepr